# Ciw Call Centre Model from JSON

In this notebook we will create a `ciw` network model from a specification in a JSON file.

The model represents a simple urgent care call centre defined in [Monks and Harper (2023)](https://openresearch.nihr.ac.uk/articles/3-48)

* A single random arrival process
* Two activities for call triage (by an operator resource) and nurse call back (by a nurse).
* Only 40% of patients require a nurse call back by default.

<img src="img/call_centre_diagram.png" alt="call_centre" width="600"/>




## 1. Imports

### 1.1 `json2ciw` imports

**We will use:**

*  `load_call_centre_model` function that loads the built in urgent care call centre JSON file.
*  `ProcessModel`: a `pydantic` schema that provides automatic validation of the JSON.
* `CiwConverter` that accepts a valid `ProcessModel` that represents a DES model and returns a `ciw` parameter `dict`.
* `multiple_replications`:runs the network model and results a `Dataframe` of replication results.
* `summarise_results`: provides a formatted table of mean results for each node in the network.

In [1]:
from json2ciw.datasets import load_call_centre_model
from json2ciw.engine import (
    CiwConverter,
    multiple_replications
)
from json2ciw.results import summarise_results

from json2ciw.schema import ProcessModel

### 1.2 Other imports

In [2]:
import ciw
import statistics
from IPython.display import JSON

## 2. Load JSON

In [3]:
json_call_centre = load_call_centre_model()

display as collapsible JSON.  Expand to view the details.

In [4]:
JSON(json_call_centre)

<IPython.core.display.JSON object>

## 3. Convert to a ProcessModel

A `ProcessModel` is a `pydantic` schema. It is independent of `ciw`. This early version will automatically validate that the JSON file is correct and that all transitions add up to 1.0 when it is created.  

In [5]:
model_instance = ProcessModel(**json_call_centre)

Transitions sum to 1.0 for all activities.


The contents of the process model can be manually inspected by a developer as follows:

In [6]:
model_instance

ProcessModel(name='Call Handling Process', description='Patient call handling process.', activities=[Activity(name='Call Triage', type='activity', resource=Resource(name='Operator', capacity=13), service_distribution=Distribution(type='triangular', parameters={'min': 5.0, 'mode': 7.0, 'max': 10.0}), arrival_distribution=Distribution(type='exponential', parameters={'rate': 0.6})), Activity(name='Nurse Consultation', type='activity', resource=Resource(name='Nurse', capacity=9), service_distribution=Distribution(type='uniform', parameters={'min': 10.0, 'max': 20.0}), arrival_distribution=None)], transitions=[Transition(source='Call Triage', target='Nurse Consultation', probability=0.4), Transition(source='Call Triage', target='Exit', probability=0.6), Transition(source='Nurse Consultation', target='Exit', probability=1.0)])

In [7]:
model_instance.display_diagram()

```mermaid
graph TD
    %% Call Handling Process: Patient call handling process.
    Arrivals_Call_Triage("Time between arrivals<br/>Exponential(λ=0.6)")
    Call_Triage["Call Triage<br/>Tri(5.0, 7.0, 10.0)"]
    Nurse_Consultation["Nurse Consultation<br/>Uniform(10.0, 20.0)"]
    Resource_Operator(("Operator<br/>(13)"))
    Resource_Nurse(("Nurse<br/>(9)"))
    Exit(["Exit"])

    Arrivals_Call_Triage --> Call_Triage
    Resource_Operator -.Seize.-> Call_Triage
    Call_Triage -.Release.-> Resource_Operator
    Resource_Nurse -.Seize.-> Nurse_Consultation
    Nurse_Consultation -.Release.-> Resource_Nurse
    Call_Triage -->|40%| Nurse_Consultation
    Call_Triage -->|60%| Exit
    Nurse_Consultation --> Exit
```

## 3. Convert to a `ciw` Network Model

In [8]:
adapter = CiwConverter(model_instance)
network_params = adapter.generate_params()

The variable `network_params` is a python `dict` that contains all the parameters that `ciw` requires for a very simple queuing model. This can be passed as keyword args to the `ciw.create_network` function.

In [9]:
network_params

{'number_of_servers': [13, 9],
 'arrival_distributions': [Exponential(rate=1.6666666666666667), None],
 'service_distributions': [Triangular(lower=5.0, mode=7.0, upper=10.0),
  Uniform(lower=10.0, upper=20.0)],
 'routing': [[0.0, 0.4], [0.0, 0.0]]}

In [10]:
# we use ciw's native `create_network` method
network = ciw.create_network(**network_params)
print(type(network))

<class 'ciw.network.Network'>


In [11]:
# Run a quick simulation to verify it works without crashing...
sim = ciw.Simulation(network)
sim.simulate_until_max_time(50)
print("Quick simulation run worked!")

Quick simulation run worked!


## 4. Run the model for multiple replications

The `multiple_replications` function returns a `DataFrame` that contains one row per activity per replication.  So if there are two nodes there are two rows for each replication.

In [12]:
# Run replications with activity/resource names
df_reps = multiple_replications(
    network, 
    model_instance, 
    num_reps=100, 
    runtime=1000
)

df_reps.head()

,rep,node_id,activity_name,resource_name,resource_capacity,arrivals,mean_wait,mean_service,utilisation,mean_Lq
0,0,1,Call Triage,Operator,13,1661,5.111651,7.325905,93.820340,8.490452
1,0,2,Nurse Consultation,Nurse,9,578,50.402823,15.076303,97.594253,29.132831
2,1,1,Call Triage,Operator,13,1618,1.549903,7.318346,91.364957,2.507743
3,1,2,Nurse Consultation,Nurse,9,589,47.393168,14.954777,98.632135,27.914576
4,2,1,Call Triage,Operator,13,1663,3.314476,7.347140,94.392726,5.511973


Filter the nodes with the usual `pandas` syntax

In [13]:
df_reps.loc[df_reps['activity_name'] == "Call Triage"].head(3)

,rep,node_id,activity_name,resource_name,resource_capacity,arrivals,mean_wait,mean_service,utilisation,mean_Lq
0,0,1,Call Triage,Operator,13,1661,5.111651,7.325905,93.820340,8.490452
2,1,1,Call Triage,Operator,13,1618,1.549903,7.318346,91.364957,2.507743
4,2,1,Call Triage,Operator,13,1663,3.314476,7.347140,94.392726,5.511973


## 5. Summarise results

If needed there is a built in function to create a `DataFrame` that contains a mean summary of all metrics across the nodes.

In [14]:
# Get summary table
summary = summarise_results(df_reps)
summary.round(1)

activity,Metric,Call Triage (Operator),Nurse Consultation (Nurse)
0,Mean arrivals,1644.2,583.8
1,Mean waiting time,3.2,44.8
2,Mean service time,7.3,15.0
3,Mean utilisation,93.1,97.9
4,Mean queue length,5.3,26.2
